# Fisher-Information Gravity: Phenomenological Solver

**Author:** Arnav Kumar
**Affiliation:** Independent Researcher, Nepal
**Reference:** Appendix B & Section VII of Manuscript

---

## 🚀 Overview
This notebook implements the numerical methods used to validate the **Fisher Information Gravity (FIG)** theory. It contains two main computational modules:

1.  **Axisymmetric Solver:** Solves the modified Poisson equation $\nabla^2 \Phi_{FI} = \alpha (\nabla^2 \chi + |\nabla \chi|^2)$ using Hankel Transforms to predict galactic rotation curves.
2.  **Cosmological Evolution:** Simulates the energy density scaling ($a^{-6}$) of the Fisher fluid to ensure consistency with Big Bang Nucleosynthesis.

In [ ]:
# Import Scientific Libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j0
from scipy.integrate import simpson

# Styling for professional plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 120

# Import Scientific Libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j0
from scipy.integrate import simpson

# Styling for professional plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 120


In [ ]:
def solve_galactic_dynamics():
    print("--- Running Axisymmetric Fisher Solver ---")

    # 1. Setup Grid (Radius in kpc)
    Rd = 2.0  # Scale length
    R_grid = np.linspace(0.1, 10.0, 100)

    # 2. Define Density & Source Term
    # n(R) ~ exp(-R/Rd)
    n = 100.0 * np.exp(-R_grid / Rd)

    # Analytical Laplacian for exponential disk in cylindrical coords
    dn_dR = - (100.0 / Rd) * np.exp(-R_grid / Rd)
    d2n_dR2 = (100.0 / Rd**2) * np.exp(-R_grid / Rd)
    laplacian_n = d2n_dR2 + (1/R_grid) * dn_dR

    # Source S = Laplacian(n)/n (From Eq 7 in Manuscript)
    S_grid = laplacian_n / n

    # 3. Hankel Transform Integration (Eq B1)
    k_grid = np.logspace(-2, 2, 200) # Wavenumber space
    phi_solution = np.zeros_like(R_grid)
    alpha = 50.0 # Effective coupling strength

    print("Computing Hankel Transforms... (This may take a moment)")
    for i, r_val in enumerate(R_grid):
        # Forward Transform: S(R) -> S_tilde(k)
        S_tilde_k = np.zeros_like(k_grid)
        for j, k_val in enumerate(k_grid):
            integrand = S_grid * j0(k_val * R_grid) * R_grid
            S_tilde_k[j] = simpson(integrand, x=R_grid)

        # Inverse Transform with Green's Function (-1/k^2)
        # We add 1e-9 to avoid division by zero at k=0
        integrand_inverse = -1.0 * (S_tilde_k / (k_grid**2 + 1e-9)) * j0(k_grid * r_val) * k_grid
        phi_solution[i] = alpha * simpson(integrand_inverse, x=k_grid)

    # 4. Compute Velocity V = sqrt(R * |dPhi/dR|)
    F_FI = -np.gradient(phi_solution, R_grid)
    V_FI = np.sqrt(np.abs(R_grid * F_FI))

    return R_grid, V_FI, Rd

# Execute Solver
R, V, Rd = solve_galactic_dynamics()

# Visualization
plt.figure(figsize=(10, 6))
plt.plot(R, V, color='#d62728', linewidth=2.5, label='Fisher Information Velocity (Emergent)')
plt.fill_between(R, 0, V, color='#d62728', alpha=0.1)
plt.axvline(x=Rd, color='gray', linestyle='--', label=f'Scale Length ($R_d={Rd}$ kpc)')

plt.title(f"Fig 1: Emergent Velocity Component (Appendix B)", fontsize=14)
plt.xlabel("Radius [kpc]", fontsize=12)
plt.ylabel("Velocity Contribution [km/s]", fontsize=12)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.show()

## 2. Cosmological Evolution

We verify the **Stiff Fluid** behavior derived in Section VII. The Fisher Information density must scale as $\rho \propto a^{-6}$ to avoid conflict with standard cosmology.

In [ ]:
# 1. Define Scale Factor (Early Universe -> Today)
a = np.logspace(-6, 0, 500)

# 2. Define Energy Densities
rho_r = 9e-5 * (a ** -4)    # Radiation
rho_m = 0.31 * (a ** -3)    # Matter
rho_FI = 1e-12 * (a ** -6)  # Fisher Information (The new theory)

# 3. Plot
plt.figure(figsize=(10, 7))
plt.loglog(a, rho_r, '--', color='orange', label='Radiation ($a^{-4}$)')
plt.loglog(a, rho_m, color='blue', label='Matter ($a^{-3}$)')
plt.loglog(a, rho_FI, color='red', linewidth=3, label='Fisher Info ($a^{-6}$)')

plt.xlabel('Scale Factor (a)', fontsize=12)
plt.ylabel('Relative Energy Density', fontsize=12)
plt.title('Fig 2: Stiff Fluid Scaling (Section VII)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, which="both", alpha=0.2)

plt.text(1e-5, 1e15, 'Fisher Information decays rapidly,\npreserving BBN predictions.',
         color='red', fontsize=10, bbox=dict(facecolor='white', alpha=0.9))

plt.show()
